In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import numpy as np
from torch import optim
%matplotlib inline
from sklearn.datasets import load_digits, fetch_openml
from sklearn.model_selection import train_test_split

In [2]:
# multi layer perceptron
# Neural Network Overview -> input layer / hidden layer(sigmoid) / output layer(sigmoid)
# forward propagation(input layer -> output layer) -> backward propagation step1(output layer -> input layer) -> backward propagation step2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# XOR probelm data
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)
# three hidden layer & output layer
model = nn.Sequential(
    nn.Linear(2, 10, bias=True),
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True),
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True),
    nn.Sigmoid(),
    nn.Linear(10, 1, bias=True),
    nn.Sigmoid()
    ).to(device)
# cost function & optimizer
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)
for epoch in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(epoch, cost.item())
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('모델의 출력값(Hypothesis): ', hypothesis.detach().cpu().numpy())
    print('모델의 예측값(Predicted): ', predicted.detach().cpu().numpy())
    print('실제값(Y): ', Y.cpu().numpy())
    print('정확도(Accuracy): ', accuracy.item())
print(list(model.parameters()))

0 0.6948983669281006
1000 0.6931379437446594
2000 0.6931172609329224
3000 0.6930763721466064
4000 0.6929103136062622
5000 0.6820822358131409
6000 0.0012996583245694637
7000 0.0004833652637898922
8000 0.00028891698457300663
9000 0.00020375008170958608
10000 0.00015648972475901246
모델의 출력값(Hypothesis):  [[1.1173233e-04]
 [9.9982893e-01]
 [9.9984252e-01]
 [1.8558851e-04]]
모델의 예측값(Predicted):  [[0.]
 [1.]
 [1.]
 [0.]]
실제값(Y):  [[0.]
 [1.]
 [1.]
 [0.]]
정확도(Accuracy):  1.0
[Parameter containing:
tensor([[-1.3060, -0.6899],
        [-0.4717, -0.5716],
        [ 0.8901,  0.9902],
        [ 0.5510,  0.2572],
        [ 4.2790, -2.4804],
        [ 0.0921,  0.9000],
        [-1.3781,  2.9243],
        [ 5.3332,  5.3745],
        [ 1.7332, -3.1540],
        [ 1.3806,  1.4408]], requires_grad=True), Parameter containing:
tensor([ 1.0597, -0.3195, -1.0387, -0.7083,  0.9477,  0.1631,  0.5985, -1.7860,
        -0.8480, -2.0306], requires_grad=True), Parameter containing:
tensor([[-0.0290,  0.0687, -0.19

In [3]:
# model save
torch.save(model.state_dict(), 'test_model.pt')

In [4]:
# definite model & load state dict
ld_model = nn.Sequential(
    nn.Linear(2, 10, bias=True),
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True),
    nn.Sigmoid(),
    nn.Linear(10, 10, bias=True),
    nn.Sigmoid(),
    nn.Linear(10, 1, bias=True),
    nn.Sigmoid()
    ).to(device)
ld_model.load_state_dict(torch.load("test_model.pt"))
ld_model.eval()

Sequential(
  (0): Linear(in_features=2, out_features=10, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=10, out_features=10, bias=True)
  (5): Sigmoid()
  (6): Linear(in_features=10, out_features=1, bias=True)
  (7): Sigmoid()
)

In [5]:
# check model
x_test = torch.FloatTensor([[0, 0], [1, 0], [1, 1], [1, 1]]).to(device)
ld_hypothesis = ld_model(x_test)
ld_predicted = (ld_hypothesis > 0.5).float()
print(ld_predicted.detach().cpu().numpy())

[[0.]
 [1.]
 [0.]
 [0.]]
